In [1]:
import pandas as pd
from pyarrow.feather import read_feather
from pathlib import Path
from spiketimes.alignment import binned_spiketrain
from spiketimes.df import spikes_df_to_binned_df, df_binned_to_bool
import numpy as np

In [2]:
data_dir = Path(".").absolute().parent / "data"
df = read_feather(str(data_dir / "spiketimes_pre.feather"))

In [3]:
df.head()

,neuron_id,spike_time_samples,experiment_name,group_name
0,276,639262,OPTO,SERT-CHR2
1,276,841590,OPTO,SERT-CHR2
2,276,1786592,OPTO,SERT-CHR2
3,276,2299466,OPTO,SERT-CHR2
4,276,2374137,OPTO,SERT-CHR2


In [4]:
df_bin = df.assign(spikes=df["spike_time_samples"].divide(30000)).pipe(spikes_df_to_binned_df, spiketimes_col="spikes", fs=1, t_start=0)

In [8]:
df_bin["has_spike"] = df_binned_to_bool(df_bin["spike_count"])

In [9]:
df_bin['has_spike2'] = np.where(df_bin['spike_count']!=0, 1, 0)
df_bin.describe()

,neuron_id,time,spike_count,has_spike,has_spike2
count,644864.000000,644864.000000,644864.000000,644864.000000,644864.000000
mean,882.093855,596.592500,0.908717,0.305137,0.305137
std,610.556599,345.666673,2.074435,0.460466,0.460466
min,276.000000,0.000000,0.000000,0.000000,0.000000
25%,491.000000,297.000000,0.000000,0.000000,0.000000
50%,720.000000,596.000000,0.000000,0.000000,0.000000
75%,915.000000,894.000000,1.000000,1.000000,1.000000
max,2373.000000,1232.000000,54.000000,1.000000,1.000000


In [17]:
(
    df_bin.groupby("neuron_id")
    .apply(lambda x: (x['has_spike'] == 0).mean()
          )
).reset_index()

neuron_id
276     0.759935
277     0.528838
278     0.446878
279     0.686131
284     0.994398
          ...   
2369    0.637037
2370    0.954733
2371    0.786480
2372    0.914403
2373    0.796373
Length: 542, dtype: float64

In [ ]:
(df_bin['has_spike'] == 0).mean()